# В этом ноутбуке происходит обучение на новых данных, к сожалению, проверка submit все также сталась на старых данных.

## Именно поэтому модели из этого ноутбка не будут совпадать с моделями в презентации

In [ ]:
%%capture

import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# %load_ext autoreload
# %autoreload 2

PIP_PATH = Path(sys.executable).parent / 'pip3'

!$PIP_PATH install \
    matplotlib \
    plotly \
    seaborn \
    pandas \
    lightgbm \
    xgboost \
    eli5 \
    shap \
    numpy \
    mlxtend \
    statsmodels \
    catboost \
    'imbalanced-learn' \
    'umap-learn[plot]'

In [ ]:
# https://www.kaggle.com/c/dont-overfit-ii/discussion/169948
DATA_DIR = Path('../input/dont-overfit-ii')
list(DATA_DIR.glob('*'))

In [ ]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime
import lightgbm as lgb
from scipy import stats
from sklearn.model_selection import (
    train_test_split, 
    StratifiedKFold, 
    KFold, 
    cross_val_score, 
    GridSearchCV, 
    RepeatedStratifiedKFold, 
    LeaveOneOut,
    StratifiedShuffleSplit
)
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, RobustScaler
import os
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score, roc_auc_score
import json
import ast
import time
from sklearn import linear_model
import eli5
from eli5.sklearn import PermutationImportance
import shap

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_selection import (
    GenericUnivariateSelect, 
    SelectPercentile, 
    SelectKBest, 
    f_classif, 
    mutual_info_classif, 
    RFE
)
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from catboost import CatBoostClassifier

In [ ]:
train = pd.read_csv(DATA_DIR / 'train.csv')
test = pd.read_csv(DATA_DIR / 'test.csv')
train.shape

In [ ]:
test.shape

In [ ]:
test.head()

<a id="de"></a>
## Data exploration

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train[train.columns[2:]].std().plot(kind='hist');
plt.title('Distribution of stds of all columns');

In [ ]:
train[train.columns[2:]].mean().plot(kind='hist');
plt.title('Distribution of means of all columns');

In [ ]:
# we have no missing values
train.isnull().any().any()

In [ ]:
print('Distributions of first 28 columns')
plt.figure(figsize=(26, 24))
for i, col in enumerate(list(train.columns)[2:30]):
    plt.subplot(7, 4, i + 1)
    plt.hist(train[col])
    plt.title(col)

In [ ]:
train['target'].value_counts()

From this overview we can see the following things:
* target is binary and has some disbalance: 36% of samples belong to 0 class;
* values in columns are more or less similar;
* columns have std of 1 +/- 0.1 (min and max values are 0.889, 1.117 respectively);
* columns have mean of 0 +/- 0.15 (min and max values are -0.2, 0.1896 respectively);

Let's have a look at correlations now!

In [ ]:
corrs = train.corr().abs().unstack().sort_values(kind="quicksort").reset_index()
corrs = corrs[corrs['level_0'] != corrs['level_1']]
corrs.tail(10)

We can see that correlations between features are lower that 0.3 and the most correlated feature with target has correlation of 0.37. So we have no highly correlated features which we could drop, on the other hand we could drop some columns with have little correlation with the target.

<a id="bm"></a>
## Basic modelling

In [ ]:
from sklearn.pipeline import Pipeline

X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']
X_test = test.drop(['id'], axis=1)
n_fold = 5
loo_folds = LeaveOneOut()
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
repeated_folds = RepeatedStratifiedKFold(n_splits=20, n_repeats=20, random_state=42)

random_folds = StratifiedShuffleSplit(n_splits=12, test_size=0.35, train_size=0.65, random_state=42)

scaler = StandardScaler()

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

X_train_splt,X_val,y_train_splt,y_val = train_test_split(X_train, y_train,
                                           test_size=.25, random_state=42, shuffle=True,stratify=y_train)

X_train = X_train.values
X_test = X_test.values

print(X_train_splt.shape ,X_val.shape, y_train_splt.shape, y_val.shape)

## Viz

In [ ]:
import random
random.seed(42)

most_corr_feats = train.corr()['target'][2:].sort_values().head(10).index.tolist()
dfp = train[["id", "target"]+most_corr_feats]
sns.set(style="ticks")

sns.pairplot(dfp.drop("id",1), hue='target');

In [ ]:
import umap 
import umap.plot

# mapper = umap.UMAP().fit(X_train)
# umap.plot.points(mapper, labels=y_train)


def draw_umap(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean', title=''):
    fit = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=n_components,
        metric=metric
    )
    u = fit.fit_transform(X_train);
    fig = plt.figure()
    if n_components == 1:
        ax = fig.add_subplot(111)
        ax.scatter(u[:,0], range(len(u)), c=y_train)
    if n_components == 2:
#         ax = fig.add_subplot(111)
#         ax.scatter(u[:,0], u[:,1], c=y_train)
        umap.plot.points(fit, labels=y_train)
    if n_components == 3:
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(u[:,0], u[:,1], u[:,2], c=y_train, s=100)
    plt.title(title, fontsize=18)
    
    
draw_umap(
    n_neighbors=20, 
    min_dist=1, 
    n_components=2, 
#     metric='cosine',
    title='UMAP viz'
)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=42)

principalComponents = pca.fit_transform(scaler.fit_transform(X_train))
principalDf = pd.DataFrame(data = principalComponents, 
                           columns = [
                               'principal component 1', 
                               'principal component 2'
                           ])

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [0, 1]
colors = ['r', 'g', ]
for target, color in zip(targets,colors):
    indicesToKeep = y_train == target
    ax.scatter(principalDf.loc[indicesToKeep, 'principal component 1']
               , principalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
pca = PCA(random_state=42)
pca.fit(scaler.fit_transform(X_train))
tmp = pd.DataFrame(pca.singular_values_)
tmp /= tmp.sum()
tmp.cumsum().plot()
plt.title('PC components value')
plt.xlabel('n_components')
plt.ylabel('% of information')
plt.show()

## Models

In [ ]:
def generate_submission_file(prediction, name):
    submission = pd.read_csv(DATA_DIR / 'sample_submission.csv')
    submission['target'] = prediction
    submission.to_csv(Path('.') / f'{name}.csv', index=False)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, r2_score, make_scorer, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances


def print_metrics(pipeline,
                  X_train=X_train_splt, X_val=X_val,
                  y_train=y_train_splt, y_val=y_val):

    perdt = pipeline.predict(X_train)
    perdv = pipeline.predict(X_val)
    cmt = confusion_matrix(y_train, perdt)
    cmv = confusion_matrix(y_val, perdv)

#     print('\nTrain Aaccuracy =',accuracy_score(y_train, perdt) ,
#           ' \nValidation Aaccuracy =',accuracy_score(y_val, perdv))
    print('---'*6)
    print('AUC train: ',roc_auc_score(y_train, perdt) )
    print('AUC Val: ',roc_auc_score(y_val,perdv) )
    print()
    print('MSE train: ', mean_squared_error(y_train, perdt))
    print('MSE Val: ',   mean_squared_error(y_val, perdv))
    print()
    print('MAE train: ', mean_absolute_error(y_train, perdt))
    print('MAE Val: ',   mean_absolute_error(y_val, perdv))
    print()    
    print('COS train: ', cosine_similarity(
        y_train.values.reshape(1, -1), perdt.reshape(1, -1))[0][0])
    print('COS Val: ',   cosine_similarity(
        y_val.values.reshape(1, -1), perdv.reshape(1, -1))[0][0])
    print()    
    print('Euclidean train: ', euclidean_distances(
        y_train.values.reshape(1, -1), perdt.reshape(1, -1))[0][0])
    print('Euclidean Val: ',   euclidean_distances(
        y_val.values.reshape(1, -1), perdv.reshape(1, -1))[0][0])
    print()    
    print('R2 train: ', r2_score(y_train, perdt))
    print('R2 Val: ',   r2_score(y_val, perdv))
    print()
    print('Confusion matrix train: \n',cmt)
    print('Confusion matrix val: \n',cmv)
    print('---'*6)


In [ ]:
def train_model(X, X_test, y, params=None, folds=random_folds, 
                model_type='sklearn', 
                averaging='usual',
                model=None,
                verbose=True,
               ):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        # print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
            
        if model_type == 'sklearn':
            model = model
            if params:
                model.set_params(**params)
#             print(model.get_params())
            model.fit(X_train, y_train)
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            
            if verbose:
                print_metrics(
                    model,
                    X_train=X_train,
                    y_train=y_train,
                    X_val=X_valid,
                    y_val=y_valid,
                )
            score = roc_auc_score(y_valid, y_pred_valid)
#             print(f'Fold {fold_n}. AUC: {score:.4f}.')
#             print('')
            
            y_pred = model.predict_proba(X_test)[:, 1]
                    
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values  
    
    prediction /= n_fold
    if verbose: 
        print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    return oof, prediction, scores

# Logistic Regression

In [ ]:
# ?linear_model.LogisticRegression

In [ ]:
import sys
import os
import warnings

# https://coderoad.ru/53784971/%D0%9A%D0%B0%D0%BA-%D0%BE%D1%82%D0%BA%D0%BB%D1%8E%D1%87%D0%B8%D1%82%D1%8C-ConvergenceWarning-%D1%81-%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E-sklearn
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regression', linear_model.LogisticRegression(
        random_state=42, 
        max_iter=500, 
        class_weight='balanced')
    )
])


def add_noise(np_array, noise_std=0.01):
#     add a bit of noise to reduce overfitting
    return np_array + np.random.normal(0, noise_std, np_array.shape)


param_grid = [
#      LASSO
    {
    'scaler': [StandardScaler(), RobustScaler()],
    'regression__solver': ['liblinear'],
    'regression__penalty' : ['l1'],
    'regression__tol' : np.logspace(-3, 0, 10),
    'regression__C' : np.logspace(1, 2, 10),
    }, 
#     RIDGE
    {
    'scaler': [StandardScaler(), RobustScaler()],
    'regression__solver': ['liblinear'],
    'regression__penalty' : ['l2'],
    'regression__tol' : np.logspace(-3, 0, 10),
    'regression__C' : np.logspace(0.5, 2, 10),
    }, 
    {
    'scaler': [StandardScaler(), RobustScaler()],
    'regression__solver': ['saga'],
    'regression__penalty' : ['elasticnet'],
    'regression__C' : np.logspace(0.5, 2, 10),
    'regression__tol' : np.logspace(-3, 0, 10),
    'regression__l1_ratio': np.logspace(-1, 0, 5)
    }
]


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    refit=True,
    n_jobs=-1,
    verbose=1,
    cv=n_fold, 
    scoring='roc_auc'
)

grid_search.fit(add_noise(X_train), y_train)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))
best_params = grid_search.best_params_
_, prediction_lr, scores_lr = train_model(add_noise(X_train), X_test, y_train, 
                                            params=best_params, 
                                            model_type='sklearn', 
                                            model=pipeline, verbose=True,
                                            folds=random_folds)
generate_submission_file(prediction_lr, 'pipeline_lr_v4')

# SVM

In [ ]:
# ?SVC

In [ ]:
from sklearn.svm import SVC

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svm', SVC(random_state=42, probability=True))
])

base_params = {
    'svm__class_weight' : ['balanced', {0:2, 1:1}],
    'svm__C': np.logspace(-1, 2, 10),
    'scaler': [
               StandardScaler(),  
               RobustScaler(),
#                KBinsDiscretizer(n_bins=25, strategy='quantile')
              ],
}

param_grid = [{
    'svm__kernel': ['linear',],
    **base_params
}, 
    {
    'svm__kernel': [ 'rbf',],
    'svm__gamma': np.logspace(-4, 0, 5),
    **base_params
}, 
    {
    'svm__kernel': ['sigmoid'],
    'svm__gamma': np.logspace(-4, 0, 5),
    'svm__coef0': np.logspace(-3, 1, 5),
    **base_params
}, 
    {

    'svm__kernel': [ 'poly'],
    'svm__degree': [3,5],
    'svm__coef0': np.logspace(-3, 0, 5),
    **base_params
}]


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    refit=True,
    n_jobs=-1,
    verbose=1,
    cv=n_fold, 
    scoring='roc_auc'
)

grid_search.fit(add_noise(X_train), y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

# print_metrics(pipeline, grid_search.best_params_)

best_params = grid_search.best_params_
_, predictions_svc, scores_svc = train_model(add_noise(X_train), X_test, y_train, 
                                     params=best_params, 
                                     model_type='sklearn', 
                                     model=pipeline, 
                                     verbose=True)
generate_submission_file(predictions_svc, 'pipeline_svc')

# Forest

In [ ]:
# ?RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('forest', RandomForestClassifier(random_state=42))
])

base_params = {
    'forest__class_weight' : [
        'balanced', 'balanced_subsample'],
    'forest__n_estimators': [150],
    'forest__criterion': ["entropy"],
    'forest__max_depth': [4, 5],
    'forest__max_samples': [15, 20, 30, None],
    'forest__ccp_alpha': [0.04, 0.1, 0.18],
    'forest__max_features': [3, 4, 5],
    'forest__max_leaf_nodes': [4, 6, 8, 10]
}


param_grid = [{
    **base_params
}]


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    refit=True,
    n_jobs=-1,
    cv=n_fold, 
    verbose=1,
    scoring='roc_auc'
)

grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

best_params = grid_search.best_params_
_, predictions_forest, scores_forest = train_model(X_train, X_test, y_train, 
                                            params=best_params, 
                                            model_type='sklearn', 
                                            model=pipeline, verbose=True)
generate_submission_file(predictions_forest, 'pipeline_forest')

## RandomUnderSampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('forest', RandomForestClassifier(random_state=42))
])

base_params = {
    'forest__class_weight' : [None, 'balanced_subsample'],
    'forest__n_estimators': [20, 50, 150],
    'forest__criterion': ["entropy"],
    'forest__max_depth': [4, 5],
    'forest__max_samples': [15, 20, 30, None],
    'forest__ccp_alpha': [0.1, 0.18],
    'forest__max_features': [3, 4, 5],
    'forest__max_leaf_nodes': [4, 6, 8, 10]
}

param_grid = [{
    **base_params
}]


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    refit=True,
    n_jobs=-1,
    cv=n_fold, 
    verbose=1,
#     scoring='roc_auc',
    scoring='accuracy'
)

grid_search.fit(X_resampled, y_resampled)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

best_params = grid_search.best_params_
_, predictions_forest_under_sample, scores_forest_under_sample = train_model(
    X_resampled, X_test, y_resampled, 
                                            params=best_params, 
                                            model_type='sklearn', 
                                            model=pipeline, verbose=True)

generate_submission_file(predictions_forest_under_sample, 'pipeline_forest_under_sample')

# LR

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regression', linear_model.LogisticRegression(
        random_state=42, 
        max_iter=500, 
        class_weight='balanced')
    )
])


def add_noise(np_array, noise_std=0.01):
#     add a bit of noise to reduce overfitting
    return np_array + np.random.normal(0, noise_std, np_array.shape)


param_grid = [
#      LASSO
    {
    'scaler': [StandardScaler(), RobustScaler()],
    'regression__solver': ['liblinear'],
    'regression__penalty' : ['l1'],
    'regression__tol' : np.logspace(-3, 0, 10),
    'regression__C' : np.logspace(1, 2, 10),
    }, 
#     RIDGE
    {
    'scaler': [StandardScaler(), RobustScaler()],
    'regression__solver': ['liblinear'],
    'regression__penalty' : ['l2'],
    'regression__tol' : np.logspace(-3, 0, 10),
    'regression__C' : np.logspace(0.5, 2, 10),
    }, 
    {
    'scaler': [StandardScaler(), RobustScaler()],
    'regression__solver': ['saga'],
    'regression__penalty' : ['elasticnet'],
    'regression__C' : np.logspace(0.5, 2, 10),
    'regression__tol' : np.logspace(-3, 0, 10),
    'regression__l1_ratio': np.logspace(-1, 0, 5)
    }
]


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    refit=True,
    n_jobs=-1,
    verbose=1,
    cv=n_fold, 
    scoring='roc_auc'
)

grid_search.fit(add_noise(X_resampled), y_resampled)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))
best_params = grid_search.best_params_
_, prediction_lr_under_sample, scores_lr_under_sample = train_model(
    add_noise(X_resampled), X_test, y_resampled, 
                                            params=best_params, 
                                            model_type='sklearn', 
                                            model=pipeline, verbose=True,
                                            folds=random_folds)
generate_submission_file(prediction_lr_under_sample, 'pipeline_lr_under_sample')

# SVC

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svm', SVC(random_state=42, probability=True))
])

base_params = {
    'svm__class_weight' : [None],
    'svm__C': np.logspace(1, 2, 10),
    'scaler': [
               StandardScaler(),  
               RobustScaler(),
#                KBinsDiscretizer(n_bins=25, strategy='quantile')
              ],
}

param_grid = [{
    'svm__kernel': ['linear',],
    **base_params
}, 
{
    'svm__kernel': [ 'rbf',],
    'svm__gamma': np.logspace(-4, 0, 5),
    **base_params
}, 
    {
    'svm__kernel': ['sigmoid'],
    'svm__gamma': np.logspace(-4, 0, 5),
    'svm__coef0': np.logspace(-3, 2, 10),
    **base_params
}, 
    {

    'svm__kernel': [ 'poly'],
    'svm__degree': [3,5],
    'svm__coef0': np.logspace(-3, 0, 5),
    **base_params
}
]


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    refit=True,
    n_jobs=-1,
    verbose=1,
    cv=n_fold, 
    scoring='roc_auc'
)

grid_search.fit(X_resampled, y_resampled)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

# print_metrics(pipeline, grid_search.best_params_)

best_params = grid_search.best_params_

_, predictions_svc_under_sample, scores_svc_under_sample = train_model(
                        X_resampled, X_test, y_resampled, 
                                     params=best_params, 
                                     model_type='sklearn', 
                                     model=pipeline, 
                                     verbose=True)
generate_submission_file(predictions_svc_under_sample, 'pipeline_svc_under_sample')

<a id="mlextend"></a>
# mutual_info_classif

In [ ]:
from sklearn.metrics import roc_auc_score, SCORERS ,auc, confusion_matrix,accuracy_score,recall_score,precision_score
from sklearn.decomposition import PCA    
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn import svm

In [ ]:
mi = mutual_info_classif(X_train, y_train, random_state=42)

In [ ]:
selectedFeature  =  np.array( mi > 0.02)
selected_cols = train.drop(['id', 'target'], axis=1).columns[selectedFeature]
print('number of selected columns',selectedFeature.sum())
print('number of selected columns', selected_cols)

In [ ]:
selected_features_train_X = train.drop(['id', 'target'], axis=1)[selected_cols].values
selected_features_test_X = test.drop(['id'], axis=1)[selected_cols].values

## RandomForest

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('forest', RandomForestClassifier(random_state=42))
])

base_params = {
    'forest__class_weight' : [None, 'balanced_subsample'],
    'forest__n_estimators': [20, 50, 150],
    'forest__criterion': ["entropy"],
    'forest__max_depth': [4, 5],
    'forest__max_samples': [15, 20, 30, None],
    'forest__ccp_alpha': [0.1, 0.18],
    'forest__max_features': [3, 4, 5],
    'forest__max_leaf_nodes': [4, 6, 8, 10]
}

param_grid = [{
    **base_params
}]


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    refit=True,
    n_jobs=-1,
    cv=n_fold, 
    verbose=1,
#     scoring='roc_auc',
    scoring='accuracy'
)

grid_search.fit(add_noise(selected_features_train_X), y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

best_params = grid_search.best_params_
_, predictions_forest_mutual_info, scores_forest_mutual_info = train_model(
    add_noise(selected_features_train_X), selected_features_test_X, y_train, 
                                            params=best_params, 
                                            model_type='sklearn', 
                                            model=pipeline, verbose=True)

generate_submission_file(predictions_forest_mutual_info, 'pipeline_forest_mutual_info')

## Logistic Regression

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regression', linear_model.LogisticRegression(
        random_state=42, 
        max_iter=500, 
        class_weight='balanced')
    )
])


param_grid = [
#      LASSO
    {
    'scaler': [StandardScaler(), RobustScaler()],
    'regression__solver': ['liblinear'],
    'regression__penalty' : ['l1'],
    'regression__tol' : np.logspace(-3, 0, 10),
    'regression__C' : np.logspace(1, 2, 10),
    }, 
#     RIDGE
    {
    'scaler': [StandardScaler(), RobustScaler()],
    'regression__solver': ['liblinear'],
    'regression__penalty' : ['l2'],
    'regression__tol' : np.logspace(-3, 0, 10),
    'regression__C' : np.logspace(0.5, 2, 10),
    }, 
    {
    'scaler': [StandardScaler(), RobustScaler()],
    'regression__solver': ['saga'],
    'regression__penalty' : ['elasticnet'],
    'regression__C' : np.logspace(0.1, 1, 10),
    'regression__tol' : np.logspace(-3, 0, 15),
    'regression__l1_ratio': np.logspace(-1, 0, 10)
    }
]


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    refit=True,
    n_jobs=-1,
    verbose=1,
    cv=n_fold, 
    scoring='roc_auc'
)

grid_search.fit(add_noise(selected_features_train_X), y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))


best_params = grid_search.best_params_
_, prediction_lr_mutual_info, scores_lr_mutual_info = train_model(add_noise(selected_features_train_X),
                                     selected_features_test_X, y_train, 
                                            params=best_params, 
                                            model_type='sklearn', 
                                            model=pipeline, verbose=True,
                                            folds=random_folds
                                    )

generate_submission_file(prediction_lr_mutual_info, 'pipeline_lr_mutual_info_v4')

In [ ]:
best_params = grid_search.best_params_
_, _, _ = train_model(add_noise(selected_features_train_X),
                                     selected_features_test_X, y_train, 
                                            params=best_params, 
                                            model_type='sklearn', 
                                            model=pipeline, verbose=True,
                                            folds=KFold(n_splits=2)
                                    )

## SVC

In [ ]:
from sklearn.svm import SVC
default_params = dict(
    class_weight='balanced',
    kernel='linear'
)

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svm', SVC(random_state=42, probability=True))
])

base_params = {
    'svm__class_weight' : ['balanced'],
    'svm__C': np.logspace(-3, 0, 10),
}

param_grid = [{
    'svm__kernel': ['linear',],
    'svm__class_weight' : ['balanced'],
    'svm__C': np.logspace(-3, 0, 10),
}, {
    'svm__kernel': [ 'rbf', 'sigmoid'],
    'svm__gamma': np.logspace(-4, 0, 10),
    'svm__class_weight' : ['balanced'],
    'svm__C': np.logspace(-3, 0, 10),
}, {
    'svm__kernel': [ 'poly',],
    'svm__degree': [2,3,],
    'svm__coef0': np.logspace(-3, 0, 10),
    'svm__class_weight' : ['balanced'],
    'svm__C': np.logspace(-3, 0, 10),
}]


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    refit=True,
    n_jobs=-1,
    cv=n_fold, 
    scoring='roc_auc',
    verbose=1,
    return_train_score=True
)


grid_search.fit(selected_features_train_X, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))


best_params = grid_search.best_params_
_, predictions_svc_mutual_info, scores_svc_mutual_info = train_model(
                                            selected_features_train_X, 
                                            selected_features_test_X, y_train, 
                                            params=best_params, 
                                            model_type='sklearn', 
                                            model=pipeline, verbose=True)
generate_submission_file(predictions_svc_mutual_info, 'pipeline_svc_mutual_info')

In [ ]:
from pprint import pprint
pprint({'svm__C': 0.0021544346900318843, 'svm__class_weight': 'balanced', 'svm__kernel': 'linear'})

<a id="model"></a>
# Fast Modelling

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
oof_gnb, prediction_gnb, scores_gnb = train_model(
    X_train, X_test, y_train,
    model_type='sklearn', model=gnb)
generate_submission_file(prediction_gnb, 'pipeline_GaussianNB')

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier()

parameter_grid = {'n_estimators': [5, 10, 20, 50, 100],
                  'learning_rate': [0.001, 0.01, 0.1, 1.0, 10.0]
                 }

grid_search = GridSearchCV(abc, param_grid=parameter_grid, cv=folds, scoring='roc_auc')
grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

abc = AdaBoostClassifier(**grid_search.best_params_)
oof_abc, prediction_abc, scores_abc = train_model(
    X_train, X_test, y_train, params=grid_search.best_params_, 
    model_type='sklearn', model=abc, verbose=True)
generate_submission_file(prediction_abc, 'pipeline_AdaBoostClassifier')

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier()

parameter_grid = {'n_estimators': [10, 50, 100, 1000],
                  'max_depth': [None, 3, 5, 15]
                 }

grid_search = GridSearchCV(etc, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))
etc = ExtraTreesClassifier()
oof_etc, prediction_etc, scores_etc = train_model(
    X_train, X_test, y_train, params=grid_search.best_params_, model_type='sklearn', 
    model=etc, verbose=True)

generate_submission_file(prediction_etc, 'pipeline_ExtraTreesClassifier')

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
gpc = GaussianProcessClassifier()
oof_gpc, prediction_gpc, scores_gpc = train_model(X_train, X_test, y_train, 
                                                   model_type='sklearn', model=gpc)

generate_submission_file(prediction_gpc, 'pipeline_GaussianProcessClassifier')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()

parameter_grid = {'n_neighbors': [2, 3, 5, 10, 20],
                  'weights': ['uniform', 'distance'],
                  'leaf_size': [5, 10, 30]
                 }

grid_search = GridSearchCV(knc, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))
knc = KNeighborsClassifier(**grid_search.best_params_)
oof_knc, prediction_knc, scores_knc = train_model(
    X_train, X_test, y_train, model_type='sklearn', model=knc)
generate_submission_file(prediction_knc, 'pipeline_KNeighborsClassifier')

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()

parameter_grid = {'alpha': [0.0001, 1, 2, 10]
                 }

grid_search = GridSearchCV(bnb, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))
bnb = BernoulliNB(**grid_search.best_params_)
oof_bnb, prediction_bnb, scores_bnb = train_model(X_train, X_test, y_train, 
                                                  model_type='sklearn', model=bnb)
generate_submission_file(prediction_bnb, 'pipeline_BernoulliNB')

In [ ]:
sgd = linear_model.SGDClassifier(eta0=1, max_iter=1000, tol=0.0001)

parameter_grid = {'loss': ['log', 'modified_huber'],
                  'penalty': ['l1', 'l2', 'elasticnet'],
                  'alpha': [0.001, 0.01],
                  'l1_ratio': [0, 0.15, 0.5, 1.0],
                  'learning_rate': ['optimal', 'invscaling', 'adaptive']
                 }

grid_search = GridSearchCV(sgd, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))
sgd = linear_model.SGDClassifier(eta0=1, tol=0.0001, **grid_search.best_params_)
oof_sgd, prediction_sgd, scores_sgd = train_model(
    X_train, X_test, y_train, model_type='sklearn', model=sgd)

generate_submission_file(prediction_sgd, 'pipeline_SGD')

In [ ]:
plt.figure(figsize=(12, 8));
scores_df = pd.DataFrame({'LogisticRegression': scores_lr})
scores_df['LogisticRegressionMutualInfo'] = scores_lr_mutual_info
scores_df['LogisticRegressionUnderSample'] = scores_lr_under_sample

scores_df['SVC'] = scores_svc
scores_df['SVCMutualInfo'] = scores_svc_mutual_info
scores_df['SVCUnderSample'] = scores_svc_under_sample

scores_df['RandomForestClassifier'] = scores_forest
scores_df['RandomForestClassifierUnderSample'] = scores_forest_under_sample
scores_df['RandomForestClassifierMutualInfo'] = scores_forest_mutual_info

scores_df['GaussianNB'] = scores_gnb
scores_df['AdaBoostClassifier'] = scores_abc
scores_df['ExtraTreesClassifier'] = scores_etc
scores_df['GaussianProcessClassifier'] = scores_gpc
scores_df['KNeighborsClassifier'] = scores_knc
scores_df['BernoulliNB'] = scores_bnb
scores_df['SGDClassifier'] = scores_sgd

sns.boxplot(data=scores_df);
plt.xticks(rotation=45);

## SGD Shap

In [ ]:
explainer = shap.LinearExplainer(sgd, X_train)
shap_values = explainer.shap_values(X_train)

shap.summary_plot(shap_values, X_train)

In [ ]:
eli5.show_weights(sgd, top=50)